## More Lions is All You Need

running scoring and voting using `llama2` model via `ollama` service

In [1]:
from IPython.display import Markdown
from timeit import default_timer as timer

In [2]:
system = "you are a helpful assistant."
instruct = "think step by step, concisely yet comprehensively explain the balance sheet. return around 200 words"

The goal is to find the best answer from 5 generations of the same questions

In [3]:
import lionagi as li

durations = []

async def explain(i):
    service = li.Services.Ollama(model='llama2')
    session = li.Session(system=system, service=service)

    # use timed call to get the runtime of the given function
    _, time_ = await li.tcall(
        func=session.chat, instruction=instruct, timing=True
    )
    
    durations.append(time_)
    return session

In [4]:
start_time = timer()

In [5]:
writer_sessions = await li.alcall([i for i in range(5)], explain)

In [6]:
run_time = timer() - start_time
print(f"\nClock time: {run_time:.2f} seconds")


Clock time: 42.93 seconds


In [7]:
import numpy as np

run_describe = f"""
Average runtime: {np.average(durations):.2f} seconds\n
Total runtime: {np.sum(durations):.2f} seconds\n
Total number of runs: {len(durations)}
"""
print(run_describe)


Average runtime: 30.89 seconds

Total runtime: 154.44 seconds

Total number of runs: 5



In [8]:
print(f"speed up { np.sum(durations) / float(run_time): .02f}")

speed up  3.60


### Get a group of critics

In [9]:
system = """
    as a highly critical professor, score an answer, only return a 
    positive integer
"""

instruction = """
    meticulously and critically give a score between 0-10 on the 
    following explaination of balance sheet.
"""

In [10]:
durations = []
results = []

async def score(context):
    service = li.Services.Ollama(model='llama2')
    
    # set up a function for one scorer to evaluate
    async def _inner(i):
        session = li.Session(system=system, service=service)
        
        out_, time_ = await li.tcall(
            func=session.chat, instruction=instruction, context=context, timing=True)
        
        durations.append(time_)
        return out_
        
    scores = await li.alcall([i for i in range(5)], _inner)        # run 5 scorers in parallel 
    
    # map the context with scores,
    out = {
        "context": context,
        "scores": scores
    }
    
    results.append(out)
    
    return (scores, context)

In [11]:
# get the responses from all previous sessions

responses = li.lcall(writer_sessions, lambda s: s.last_response_content['response'])

this is 25 runs in total across all responses,
so will be pretty slow

to speed up, reduce the number of responses


In [12]:
start_time = timer()

In [13]:
outputs = await li.alcall(responses, score)

In [14]:
run_time = timer() - start_time
print(f"\nClock time: {run_time:.2f} seconds")


Clock time: 191.95 seconds


In [15]:
scores = [o[0] for o in outputs]
contexts = [o[1] for o in outputs]

In [16]:
run_time = f"""
Average runtime: {np.average(durations):.2f} seconds\n
Total runtime: {np.sum(durations):.2f} seconds\n
Total number of runs: {len(durations)}
"""
print(run_time)


Average runtime: 135.40 seconds

Total runtime: 3384.90 seconds

Total number of runs: 25



In [17]:
avgs = []
for i in scores:
    z = []
    for j in i:
        try:
            a = li.str_to_num(j)
            z.append(a)
        except:
            pass
    avgs.append(np.average(z))

print(avgs)

[8.0, 8.0, 6.2, 8.4, 8.0]


In [18]:
high_index = np.where(np.array(avgs) == max(avgs))
low_index = np.where(np.array(avgs) == min(avgs))

In [19]:
high_index

(array([3]),)

In [20]:
low_index

(array([2]),)

### High Score

In [21]:
print(f"High score: {max(avgs)}")
Markdown(contexts[4])

High score: 8.4



As a helpful assistant, I'd be happy to explain the balance sheet in a clear and concise manner!

The balance sheet is a financial statement that provides a snapshot of a company's assets, liabilities, and equity at a specific point in time. It is used to understand the financial position of a company and make informed decisions about its operations.

Here are the key components of a balance sheet:

1. Assets: These are the resources that a company owns or controls, such as cash, accounts receivable, inventory, property, plant, and equipment.
2. Liabilities: These are the debts or obligations that a company owes to others, such as loans, accounts payable, and taxes owed.
3. Equity: This represents the ownership interest in a company, including shares of stock held by shareholders and retained earnings.
4. Net worth: This is the difference between a company's assets and liabilities, and represents the true value of the company.
5. Current ratio: This is a measure of a company's ability to pay its short-term debts, calculated by dividing current assets by current liabilities.
6. Debt-to-equity ratio: This measures a company's level of debt relative to its equity, and can help investors assess the level of risk in investing in a company.
7. Return on equity (ROE): This measures a company's net income as a percentage of its shareholders' equity, and helps investors evaluate a company's profitability.

By analyzing these components, investors and analysts can gain insights into a company's financial health and make informed decisions about investing or lending to the company.

### Low score

In [22]:
print(f"Low score: {min(avgs)}")
Markdown(contexts[1])

Low score: 6.2



As a helpful assistant, I'd be happy to explain the balance sheet in a clear and concise manner!

The balance sheet is a financial statement that provides a snapshot of a company's assets, liabilities, and equity at a specific point in time. It is used to help investors, creditors, and other stakeholders understand a company's financial position and make informed decisions.

The balance sheet consists of three main sections:

1. Assets: This section lists all the resources that the company owns or controls, such as cash, accounts receivable, inventory, property, plant, and equipment.
2. Liabilities: This section lists all the debts and obligations that the company owes to others, such as loans, accounts payable, and taxes owed.
3. Equity: This section shows the ownership structure of the company, including the amount of shares issued and outstanding, retained earnings, and any other reserves or funds.

The balance sheet is important because it provides a comprehensive picture of a company's financial position at a given time. By analyzing the balance sheet, investors can determine whether a company has sufficient assets to cover its liabilities and ensure that it can meet its financial obligations. Creditors can also use the balance sheet to assess the creditworthiness of a company and make informed lending decisions.

In conclusion, the balance sheet is a fundamental financial statement that provides valuable information about a company's financial position. By understanding the balance sheet, investors and creditors can make more informed decisions about investing in or lending to a particular company.

### Others

In [23]:
print(f"High score: {avgs[0]}")
Markdown(contexts[0])

High score: 8.0



As a helpful assistant, I'd be delighted to explain the balance sheet in a clear and concise manner! Here it goes:

The balance sheet is a financial statement that provides an overview of a company's assets, liabilities, and equity at a specific point in time. It is a snapshot of the company's financial position, showing how much money the company has (assets) and how much it owes (liabilities) to external parties. The balance sheet is divided into two sides: the asset side and the liability side.

On the asset side, the balance sheet lists all the resources owned by the company, such as cash, inventory, property, and equipment. These assets are valued at their original cost or their current market value, whichever is lower. The total value of the asset side should equal the total value of the liability side, which represents the amount owed to external parties.

On the liability side, the balance sheet lists all the debts and obligations of the company, including loans, accounts payable, taxes payable, and wages payable. These liabilities are also valued at their original cost or their current market value.

The difference between the asset side and the liability side is called equity, which represents the ownership interest in the business. Equity can include shares issued to investors, retained earnings (profits), and other reserves.

In summary, the balance sheet provides a comprehensive picture of a company's financial position by listing its assets, liabilities, and equity. It is an essential tool for investors, creditors, and management to evaluate the financial health of a business.

In [24]:
print(f"High score: {avgs[2]}")
Markdown(contexts[2])

High score: 6.2



As a helpful assistant, I'm happy to provide you with a clear and concise explanation of the balance sheet!

The balance sheet is a financial statement that provides a snapshot of a company's assets, liabilities, and equity at a specific point in time. It helps investors, creditors, and other stakeholders understand a company's financial position and make informed decisions.

Here are the main components of a balance sheet:

1. Assets: These are the resources that a company owns or controls, such as cash, inventory, property, equipment, and investments.
2. Liabilities: These are the debts or obligations that a company owes to others, such as loans, accounts payable, and taxes owing.
3. Equity: This represents the ownership interest in a company, including shareholder equity and retained earnings.

The balance sheet is organized into these three main sections, with each section presented in a specific format. The assets are listed first, followed by the liabilities and equity. Within each section, the items are listed in a specific order, such as cash before accounts receivable and inventory before property.

When analyzing a balance sheet, investors and creditors typically look at the following ratios:

1. Current Ratio: This compares the current assets to current liabilities to determine if the company has enough liquidity to pay its bills.
2. Debt-to-Equity Ratio: This measures the proportion of debt to equity in a company's capital structure.
3. Return on Equity (ROE): This calculates how well a company is using its equity to generate profits.
4. Acid Test Ratio: This measures a company's ability to pay its short-term liabilities with its liquid assets.

By analyzing these ratios and reviewing the balance sheet components, investors can gain valuable insights into a company's financial health and make informed investment decisions.

In [25]:
print(f"High score: {avgs[3]}")
Markdown(contexts[3])

High score: 8.4



As a helpful assistant, I'd be delighted to explain the balance sheet in a clear and concise manner!

A company's balance sheet is a financial statement that provides a snapshot of its financial position at a specific point in time. It lists the company's assets, liabilities, and equity, which are the three main components of its financial position.

Assets: These are the resources owned by the company that have economic value. Examples include cash, inventory, property, plant, and equipment (PP&E), and investments.

Liabilities: These are the debts or obligations that the company owes to others. Examples include loans, accounts payable, and taxes payable.

Equity: This represents the ownership interest in the company. It includes shares held by shareholders, retained earnings (the company's profits), and other reserves.

The balance sheet is important because it helps investors, creditors, and other stakeholders understand a company's financial position and make informed decisions. For example, if a company has a high level of assets relative to its liabilities, it may be considered financially stable. On the other hand, if a company has a high level of liabilities relative to its assets, it may be considered financially leveraged.

In conclusion, the balance sheet provides a comprehensive overview of a company's financial position at a given point in time, and is an essential tool for investors, creditors, and other stakeholders to evaluate the company's financial health.